# Primary Gas Fluid Properties Fits `post_process.ipynb`

Suppose we have measurements for temperature pressure, and want to determine an unknown fluid property $Y$. We will measure these values within a specific range:

Fluid temperature, $T\in[10, 30]$  °C 

Fluid pressure, $P\in[0.95, 120]$ bar(a)

## 1 Transform Features (Poly Surface)

To enhance the model's ability to represent an unknown fluid property effectively, we map the measurands into a new vector space. This transformation involves converting the input vector $X$ to a polynomial $n^{th}$ degree surface form, denoted as $\Phi$.

Given the vector $X = [T, P]$, where $T$ and $P$ represent temperature and pressure respectively, the transformation can be described by:

$$
X \mapsto \Phi(T, P)
$$

$$
\Phi(T, P) = \{ T^i P^j \mid i \geq 0, j \geq 0, i + j \leq n \}
$$

Here, $X$ belongs to $\mathbb{R}^2$ and $\Phi$ to $\mathbb{R}^{\binom{2+n}{n}}$, reflecting the increased dimensionality of the feature space post-transformation.

The optimal selection of the polynomial degree $n$ for the model is determined using a hyperparameter grid search method or a similar approach.

## 2 Linear Regression Model

Once we have transformed our measurands into the new feature space using $\Phi(T, P)$, the next step is to establish a relationship between these features and the unknown fluid property, $Y$. This relationship is modeled through a linear regression approach.

We seek to find a vector of coefficients $\vec{\beta} \in \mathbb{R}^{\binom{2+n}{n}}$, which best maps the functional form $\Phi(T, P)$ to $Y$. The model can be expressed as:

$$
Y = \Phi(T,P) \vec{\beta} + \epsilon
$$

Here, $\epsilon$ represents some unknown error, arising from the assumptions inherent in the model selection and the limitations of the linear approach. This error term captures the discrepancy between the predicted and actual values of $Y$.

## 3 Fluid Density Fit

To enhance the accuracy of density calculations beyond what is achievable with a naive linear regression model, we consider splitting the fluid density into two components. The first component is the ideal gas density, $\rho_{ideal}$, and the second accounts for deviations from ideality using the compressibility factor, $Z$.

The ideal gas density is defined by the equation:
$$
\rho_{ideal} = \frac{P}{RT}
$$
where $P$ is the pressure, $T$ is the temperature, and $R$ is the specific gas constant.

However, since we do not have direct measurements of $Z$, we use the polynomial model $\Phi(T,P)\vec{\beta}$ to approximate $\hat{Z}\approx Z$.

With $\hat{Z}$ estimated, we can then approximate the fluid density, accounting for non-ideal behavior, as follows:
$$
\hat{\rho} = \frac{P}{RT} \cdot \left(\Phi(T,P) \vec{\beta}\right)^{-1}
$$

This formulation allows us to estimate the density as a function of pressure, temperature, and the specific gas constant, while incorporating adjustments based on the calculated compressibility factor from our model.

In [1]:
"""HELPER FUNCTIONS"""
import pandas as pd
from tqdm import tqdm
from refprop.utils import generate_temperature_pressure_samples
from refprop.refprop import RefpropInterface

def rename_columns(df:pd.DataFrame):
    df["Temperature (C)"] = df["T"] - 273.15  # K to C
    df["Pressure (bar)"] = df["P"] / 1e5  # Pa to bar
    df.drop(columns=["T", "P"], inplace=True)
    df.rename(columns={
        "Z": "Compressibility",
        "D": "Density (kg/m3)",
        "VIS": "Viscosity (Pa*s)",
        "ISENK": "Isentropic Exponent",
        "CSTAR": "Critical Flow Factor"
    }, inplace=True)
    df
    return df


def get_fluid_property(doe:pd.DataFrame, hFld:str, hOut:str, z:list, show_progress:bool=False):
    if show_progress:
        progress_bar = tqdm
    else:
        progress_bar = lambda x: x
    refprop = RefpropInterface(r"T:\Joseph McGovern\Code\GitHub\refprop-dotnet\refprop", hFld)
    df = pd.DataFrame()
    for index, sample in progress_bar(doe.iterrows(), total=len(doe)):
        refprop_output = refprop.refprop2dll(
            hFld,
            "TP",  # Input string of properties (Temperature and Pressure)
            hOut,  # Output properties to be calculated
            21,  # mass base SI units
            0,
            sample["Temperature [K]"],  # Temperature in K
            sample["Pressure [Pa]"],  # Pressure in Pa
            z  # composition array
        )
        df = pd.concat([df, pd.DataFrame(refprop_output, index=[index])])
    df["T"] = doe["Temperature [K]"]
    df["P"] = doe["Pressure [Pa]"]
    df = rename_columns(df)
    cols = df.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    df = df[cols]
    return df

### Hydrogen

In [3]:
from fit_fluid_properties import fit_gas_density

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "HYDROGEN PROPERTIES.xlsx",
    "reference_fluid": "HYDROGEN",
    "model_poly_degree": 4,
}

df_density, functional_form_text_density = fit_gas_density(params)

print(functional_form_text_density)
df_density.describe()

P * 10**5 / 4124.478823 / (T + 273.15) / (0.9999996503341386 + 0.0*T**0*P**0 + 1.5624346941144112e-06*T**1*P**0 + 0.0006142456861551616*T**0*P**1 + -7.017436144301784e-08*T**2*P**0 + -1.1696586611752336e-06*T**1*P**1 + 2.2805774858375242e-07*T**0*P**2 + 8.806008765633321e-10*T**3*P**0 + 1.0865006216613406e-09*T**2*P**1 + -3.7173626537650946e-09*T**1*P**2 + 1.0184750887635526e-10*T**0*P**3 + -2.093171032044503e-12*T**4*P**0 + -7.192747915930752e-12*T**3*P**1 + 1.557805053574199e-11*T**2*P**2 + 2.5086146793082366e-12*T**1*P**3 + -7.116163927959088e-13*T**0*P**4)


,Temperature (C),Pressure (bar),Compressibility,Compressibility_hat,Density (kg/m3),Density_hat (kg/m3),Density Error (kg/m3),Density Relative Error (%)
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,2.500000e+05,250000.000000
mean,27.500000,100.507625,1.060590,1.060590,7.515583,7.515591,7.455737e-06,0.000099
std,13.016414,58.143236,0.035694,0.035694,4.195217,4.195221,1.416794e-05,0.000176
min,5.000000,0.002000,1.000001,1.000000,0.000150,0.000150,-2.753919e-05,-0.000917
25%,16.250000,50.254812,1.029582,1.029582,3.928491,3.928490,-6.662032e-07,-0.000016
50%,27.500000,100.507625,1.060040,1.060040,7.646113,7.646124,4.292219e-06,0.000098
75%,38.750000,150.760437,1.091235,1.091235,11.148858,11.148877,1.395927e-05,0.000215
max,50.000000,201.013250,1.130554,1.130554,15.498325,15.498342,1.031380e-04,0.000731


### Nitrogen

In [1]:
from fit_fluid_properties import fit_gas_density

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "NITROGEN PROPERTIES.xlsx",
    "reference_fluid": "NITROGEN",
    "model_poly_degree": 5,
}

df_density, functional_form_text_density = fit_gas_density(params)
print(functional_form_text_density)
df_density.describe()

P * 10**5 / 296.8 / (T + 273.15) / (1.000035662890171 + 0.0*T**0*P**0 + -2.954632225794376e-06*T**1*P**0 + -0.00045637203444673315*T**0*P**1 + -8.697701655343783e-08*T**2*P**0 + 1.2674803940818579e-05*T**1*P**1 + 2.6573142961333596e-06*T**0*P**2 + 7.173163610972774e-09*T**3*P**0 + -1.0100057215477905e-07*T**2*P**1 + -2.6008427814149528e-08*T**1*P**2 + 5.552035247385304e-09*T**0*P**3 + -1.117894071746344e-10*T**4*P**0 + 4.751663897549653e-10*T**3*P**1 + 2.78256785015994e-10*T**2*P**2 + -9.990672817986676e-11*T**1*P**3 + -1.7261044978702176e-11*T**0*P**4 + 4.408684955178172e-13*T**5*P**0 + -4.951617533550193e-13*T**4*P**1 + -1.451168984779594e-12*T**3*P**2 + 1.6951194948365398e-13*T**2*P**3 + 2.1533239789599566e-13*T**1*P**4 + 6.219216171920875e-15*T**0*P**5)


,Temperature (C),Pressure (bar),Compressibility,Compressibility_hat,Density (kg/m3),Density_hat (kg/m3),Density Error (kg/m3),Density Relative Error (%)
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,27.500000,100.507625,1.012981,1.012981,110.429395,110.430845,0.001450,0.001313
std,13.016414,58.143236,0.019632,0.019632,62.662838,62.663660,0.000979,0.000411
min,5.000000,0.002000,0.986098,0.986098,0.002085,0.002085,-0.001378,-0.000704
25%,16.250000,50.254812,0.998112,0.998113,56.386593,56.387763,0.000723,0.001044
50%,27.500000,100.507625,1.005454,1.005451,111.992439,111.993440,0.001368,0.001318
75%,38.750000,150.760437,1.026218,1.026219,164.689311,164.691911,0.001999,0.001513
max,50.000000,201.013250,1.071206,1.071228,234.212193,234.213878,0.006386,0.003297


### Methane

In [23]:
from fit_fluid_properties import fit_gas_density

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "METHANE PROPERTIES.xlsx",
    "reference_fluid": "METHANE",
    "model_poly_degree": 6,
}

df_density, functional_form_text_density = fit_gas_density(params)

print(functional_form_text_density)
df_density.describe()

P * 10**5 / 518.3 / (T + 273.15) / (0.9997976211346609 + 3.5660253965712377e-07*T**0*P**0 + 2.3239983384877132e-05*T**1*P**0 + -0.0023325467013215876*T**0*P**1 + -1.4898601522655757e-06*T**2*P**0 + 2.675889234165983e-05*T**1*P**1 + -3.832669744657767e-07*T**0*P**2 + 1.0799131831086862e-07*T**3*P**0 + -2.2717474540382608e-07*T**2*P**1 + 2.0165594491489958e-07*T**1*P**2 + -1.2604044468870253e-08*T**0*P**3 + -3.5713362064976466e-09*T**4*P**0 + 3.190587706788658e-09*T**3*P**1 + -3.250947336223443e-09*T**2*P**2 + -1.1297755302540863e-09*T**1*P**3 + 5.023332113330951e-10*T**0*P**4 + 4.765527177434857e-11*T**5*P**0 + -9.52900045387211e-12*T**4*P**1 + -9.90440765469433e-12*T**3*P**2 + 3.20763769835351e-11*T**2*P**3 + -4.9406138284315645e-12*T**1*P**4 + -1.669415512746137e-12*T**0*P**5 + -2.2284530452486992e-13*T**6*P**0 + -1.0488525235437499e-13*T**5*P**1 + 1.4197529348385754e-13*T**4*P**2 + -6.553532214677605e-14*T**3*P**3 + -5.0236366313492775e-14*T**2*P**4 + 1.9623560419581697e-14*T**1*P**5

,Temperature (C),Pressure (bar),Compressibility,Compressibility_hat,Density (kg/m3),Density_hat (kg/m3),Density Error (kg/m3),Density Relative Error (%)
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,27.500000,100.507625,0.874808,0.874808,76.891571,76.887178,-0.004394,-0.005694
std,13.016414,58.143236,0.061620,0.061619,47.321324,47.318581,0.005919,0.004760
min,5.000000,0.002000,0.755730,0.755550,0.001194,0.001194,-0.043621,-0.030591
25%,16.250000,50.254812,0.828672,0.828666,34.989153,34.986794,-0.005467,-0.008148
50%,27.500000,100.507625,0.867576,0.867572,75.331139,75.327470,-0.003385,-0.005616
75%,38.750000,150.760437,0.921702,0.921719,117.439957,117.435219,-0.001297,-0.003267
max,50.000000,201.013250,0.999997,1.000170,180.695203,180.656038,0.031440,0.018977


### CO2

In [6]:
from fit_fluid_properties import fit_gas_density

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "CO2 PROPERTIES.xlsx",
    "reference_fluid": "CO2",
    "model_poly_degree": 7,
}

df_density, functional_form_text_density = fit_gas_density(params)

print(functional_form_text_density)
df_density.describe()

P * 10**5 / 189 / (T + 273.15) / (0.9999980651266834 + 5.862829924609642e-07*T**0*P**0 + 4.2072125413760666e-07*T**1*P**0 + -0.006617931878264571*T**0*P**1 + -5.987132640031574e-08*T**2*P**0 + 7.946881626716114e-05*T**1*P**1 + -3.369661708553546e-05*T**0*P**2 + 5.094688039224039e-09*T**3*P**0 + -6.178178045264415e-07*T**2*P**1 + 9.395696014310764e-07*T**1*P**2 + -3.824633901367328e-07*T**0*P**3 + -2.866926097755248e-10*T**4*P**0 + 4.327600636412701e-09*T**3*P**1 + -1.4276742004843114e-08*T**2*P**2 + 1.7632525528287498e-08*T**1*P**3 + -5.71363825229e-09*T**0*P**4 + 1.0168438205512067e-11*T**5*P**0 + -5.78003074983546e-11*T**4*P**1 + 2.5092643976493e-10*T**3*P**2 + -5.464521844253003e-10*T**2*P**3 + 4.2251667623234286e-10*T**1*P**4 + -1.0226188058792995e-10*T**0*P**5 + -1.9470417676657067e-13*T**6*P**0 + 1.3817759575267903e-12*T**5*P**1 + -6.233360768057126e-12*T**4*P**2 + 1.55348927588852e-11*T**3*P**3 + -1.8089824607200053e-11*T**2*P**4 + 9.465213365436274e-12*T**1*P**5 + -1.7023995516

,Temperature (C),Pressure (bar),Compressibility,Compressibility_hat,Density (kg/m3),Density_hat (kg/m3),Density Error (kg/m3),Density Relative Error (%)
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,30.000000,15.507625,0.920280,0.920280,30.479876,30.467629,-0.012248,-0.040181
std,11.570146,8.970117,0.050079,0.050079,18.894837,18.887243,0.007593,0.000076
min,10.000000,0.002000,0.780790,0.780807,0.003276,0.003275,-0.031414,-0.042307
25%,20.000000,7.754812,0.881929,0.881929,14.037290,14.031657,-0.018538,-0.040220
50%,30.000000,15.507625,0.923691,0.923691,29.335704,29.323895,-0.011788,-0.040182
75%,40.000000,23.260438,0.962950,0.962950,46.142697,46.124141,-0.005640,-0.040144
max,50.000000,31.013250,0.999992,1.000001,74.252165,74.220751,-0.000001,-0.039694


## 4 Fluid Dynamic Viscosity Fit

Similarly to the density calculation, to enhance the precision of dynamic viscosity calculations beyond what a naive linear regression model could offer, we separate the viscosity equation into two parts. The first is an idealized part calculated using Sutherland's Law, $\mu_{ideal}$, and the second is a compressible component calculated using a nonlinear function of temperature and pressure, $f(T, P)$.

The ideal part of the viscosity, $\mu_{ideal}$, is given by:
$$
\mu_{ideal} = \mu_0 \left(\frac{T}{T_0}\right)^{3/2} \frac{T_0 + S}{T + S}
$$
where $T_0$ is a reference temperature, $\mu_0$ is the reference viscosity at $T_0$, and $S$ is the Sutherland temperature for the fluid.

Given that we have measurements of $T$ and $P$, we can employ the polynomial model $\Phi(T, P)\vec{\beta}$ to approximate the compressible part of the equation:
$$
f(T, P) \approx \Phi(T, P)\vec{\beta}
$$

Consequently, the final approximation of the viscosity can be expressed as:
$$
\hat{\mu} = \mu_{ideal} \cdot \hat{f}(T, P) = \mu_0 \left(\frac{T}{T_0}\right)^{3/2} \frac{T_0 + S}{T + S} \cdot \Phi(T, P)\vec{\beta}
$$

This formulation then allows us to estimate the dynamic viscosity as a function of pressure, temperature, and known constants, while incorporating adjustments based on the real gas behaviours from our polynomial model.

### Hydrogen

In [3]:
"""Viscosity Fit Example Code with inputs for temperature and pressure in degrees Celsius and bar(a), respectively."""

from fit_fluid_properties import fit_gas_viscosity

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "HYDROGEN PROPERTIES.xlsx",
    "reference_fluid": "HYDROGEN",
    "model_poly_degree": 3,
}

df_viscosity, functional_form_text_viscosity = fit_gas_viscosity(params)

print(functional_form_text_viscosity)
df_viscosity.describe()

8.411e-06 * ((T + 273.15) / 273)**(3/2) * (273 + 97) / (T + 273.15 + 97) * (0.9953148274347787 + 0.0*T**0*P**0 + -0.0002501879569604264*T**1*P**0 + 9.393854211656465e-05*T**0*P**1 + 1.8994904480236088e-06*T**2*P**0 + -1.5943132412625415e-06*T**1*P**1 + 9.127755307756316e-07*T**0*P**2 + -9.001903491318135e-09*T**3*P**0 + 1.0376487435675505e-08*T**2*P**1 + -3.719313518586319e-09*T**1*P**2 + -7.636829217508127e-10*T**0*P**3)


,Temperature (C),Pressure (bar),Viscosity (Pa*s),Viscosity_hat (Pa*s),Viscosity Error (Pa*s),Viscosity Relative Error (%)
count,250000.000000,250000.000000,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05
mean,27.500000,100.507625,9.087843e-06,9.087843e-06,5.404794e-19,3.871976e-08
std,13.016414,58.143236,2.754062e-07,2.754062e-07,1.720013e-10,1.900235e-03
min,5.000000,0.002000,8.482137e-06,8.481129e-06,-1.008564e-09,-1.189045e-02
25%,16.250000,50.254812,8.871032e-06,8.870979e-06,-1.105977e-10,-1.217413e-03
50%,27.500000,100.507625,9.092338e-06,9.092332e-06,-4.146712e-12,-4.570568e-05
75%,38.750000,150.760437,9.311970e-06,9.311989e-06,1.039306e-10,1.142268e-03
max,50.000000,201.013250,9.708521e-06,9.707628e-06,4.868119e-10,5.696888e-03


### Nitrogen

In [1]:
"""Viscosity Fit Example Code with inputs for temperature and pressure in degrees Celsius and bar(a), respectively."""

from fit_fluid_properties import fit_gas_viscosity

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "NITROGEN PROPERTIES.xlsx",
    "reference_fluid": "NITROGEN",
    "model_poly_degree": 5,
}

df_viscosity, functional_form_text_viscosity = fit_gas_viscosity(params)

print(functional_form_text_viscosity)
df_viscosity.describe()

1.663e-05 * ((T + 273.15) / 273)**(3/2) * (273 + 107) / (T + 273.15 + 107) * (0.9986751487520209 + 0.0*T**0*P**0 + 2.766711499647596e-05*T**1*P**0 + 0.0008554650432127511*T**0*P**1 + -3.8074998614309534e-08*T**2*P**0 + -6.008950401748402e-06*T**1*P**1 + 6.507984141452443e-06*T**0*P**2 + 1.2819974558241625e-09*T**3*P**0 + 4.775077081300045e-08*T**2*P**1 + -9.059226706436514e-08*T**1*P**2 + 2.1897977983390376e-09*T**0*P**3 + 1.340726624276883e-10*T**4*P**0 + -6.357295424516017e-10*T**3*P**1 + 7.358025465536181e-10*T**2*P**2 + 8.302399868308518e-11*T**1*P**3 + -6.120477108281297e-11*T**0*P**4 + -1.8406940525754158e-12*T**5*P**0 + 4.10168801080302e-12*T**4*P**1 + -2.189470108341104e-12*T**3*P**2 + -9.145110054812345e-13*T**2*P**3 + 1.7993101701917506e-13*T**1*P**4 + 9.574457586892897e-14*T**0*P**5)


,Temperature (C),Pressure (bar),Viscosity (Pa*s),Viscosity_hat (Pa*s),Viscosity Error (Pa*s),Viscosity Relative Error (%)
count,249999.000000,249999.000000,249999.000000,249999.000000,2.499990e+05,2.499990e+05
mean,27.500048,100.507283,0.000020,0.000020,4.255466e-16,1.354952e-08
std,13.016418,58.143100,0.000002,0.000002,2.344740e-10,1.176629e-03
min,5.000000,0.002000,0.000017,0.000017,-1.103303e-09,-4.792342e-03
25%,16.272545,50.154105,0.000019,0.000019,-1.273302e-10,-6.303441e-04
50%,27.545090,100.306211,0.000020,0.000020,4.134106e-12,2.068095e-05
75%,38.772545,150.659730,0.000022,0.000022,1.255213e-10,6.234354e-04
max,50.000000,201.013250,0.000024,0.000024,1.073756e-09,6.371296e-03


### Methane

In [5]:
"""Viscosity Fit Example Code with inputs for temperature and pressure in degrees Celsius and bar(a), respectively."""

from fit_fluid_properties import fit_gas_viscosity

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "METHANE PROPERTIES.xlsx",
    "reference_fluid": "METHANE",
    "model_poly_degree": 6,
}

df_viscosity, functional_form_text_viscosity = fit_gas_viscosity(params)

print(functional_form_text_viscosity)
df_viscosity.describe()

1.49e-05 * ((T + 273.15) / 273.15)**(3/2) * (273.15 + 169) / (T + 273.15 + 169) * (0.687343118089021 + -1.3817707397079688e-07*T**0*P**0 + -1.1692852295528582e-05*T**1*P**0 + 0.0009142586647460186*T**0*P**1 + 1.280641695339874e-06*T**2*P**0 + -1.344947011551101e-06*T**1*P**1 + 7.072459474211324e-06*T**0*P**2 + -8.525075222042702e-09*T**3*P**0 + -1.8743701179971104e-07*T**2*P**1 + -5.889347647093578e-08*T**1*P**2 + 7.502401926029263e-08*T**0*P**3 + 1.083723683379424e-09*T**4*P**0 + -1.3028302549614355e-10*T**3*P**1 + 5.906392404956118e-09*T**2*P**2 + -4.144093739961043e-09*T**1*P**3 + 1.0674588797208151e-10*T**0*P**4 + -3.4167292631228366e-11*T**5*P**0 + 6.160901831016144e-11*T**4*P**1 + -9.051006367652478e-11*T**3*P**2 + 2.1189251955192933e-11*T**2*P**3 + 1.57838374481394e-11*T**1*P**4 + -2.4707777664176e-12*T**0*P**5 + 2.261823647965134e-13*T**6*P**0 + -2.3455924128032296e-13*T**5*P**1 + 2.5958414684205864e-14*T**4*P**2 + 2.4458313881182156e-13*T**3*P**3 + -1.3299617364054053e-13*T**2

,Temperature (C),Pressure (bar),Viscosity (Pa*s),Viscosity_hat (Pa*s),Viscosity Error (Pa*s),Viscosity Relative Error (%)
count,250000.000000,250000.000000,250000.000000,250000.000000,2.500000e+05,250000.000000
mean,27.500000,100.507625,0.000014,0.000014,2.517112e-14,0.000002
std,13.016414,58.143236,0.000002,0.000002,1.257076e-09,0.008929
min,5.000000,0.002000,0.000010,0.000010,-7.112012e-09,-0.037506
25%,16.250000,50.254812,0.000012,0.000012,-6.621790e-10,-0.004719
50%,27.500000,100.507625,0.000014,0.000014,2.007078e-11,0.000137
75%,38.750000,150.760437,0.000016,0.000016,6.061249e-10,0.004437
max,50.000000,201.013250,0.000021,0.000021,5.724350e-09,0.049447


### CO2

In [6]:
from fit_fluid_properties import fit_gas_viscosity

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "CO2 PROPERTIES.xlsx",
    "reference_fluid": "CO2",
    "model_poly_degree": 6,
}

df_viscosity, functional_form_text_viscosity = fit_gas_viscosity(params)

print(functional_form_text_viscosity)
df_viscosity.describe()

1.37e-05 * ((T + 273.15) / 273)**(3/2) * (273 + 222) / (T + 273.15 + 222) * (0.9996501985093463 + -4.371197956202665e-10*T**0*P**0 + 0.00012074979588244215*T**1*P**0 + 0.0005758414612009825*T**0*P**1 + 6.148449267322795e-07*T**2*P**0 + -4.11263766775305e-06*T**1*P**1 + 2.345632525368664e-05*T**0*P**2 + -7.3137244355340165e-09*T**3*P**0 + 1.887224296586453e-08*T**2*P**1 + -2.687641983152508e-07*T**1*P**2 + 3.005587554294305e-07*T**0*P**3 + 1.248284878332085e-10*T**4*P**0 + -2.7874862834305832e-11*T**3*P**1 + 1.7166155262269721e-09*T**2*P**2 + -7.1419991488371595e-09*T**1*P**3 + 4.671978928460646e-09*T**0*P**4 + -2.7836039380737283e-12*T**5*P**0 + 1.2929165563637432e-11*T**4*P**1 + -7.039148153353431e-11*T**3*P**2 + 2.461716809985011e-10*T**2*P**3 + -3.1325817144128873e-10*T**1*P**4 + 1.116113277214542e-10*T**0*P**5 + 2.8836376103798245e-14*T**6*P**0 + -2.9147433707084336e-13*T**5*P**1 + 1.6823503277052423e-12*T**4*P**2 + -5.2581570621581565e-12*T**3*P**3 + 8.062440212997543e-12*T**2*P**

,Temperature (C),Pressure (bar),Viscosity (Pa*s),Viscosity_hat (Pa*s),Viscosity Error (Pa*s),Viscosity Relative Error (%)
count,250000.000000,250000.000000,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05
mean,30.000000,15.507625,1.538093e-05,1.538093e-05,-2.703329e-17,2.310046e-10
std,11.570146,8.970117,5.658921e-07,5.658921e-07,2.496254e-11,1.633719e-04
min,10.000000,0.002000,1.419329e-05,1.419329e-05,-5.009602e-10,-3.363860e-03
25%,20.000000,7.754812,1.491941e-05,1.491940e-05,-1.444321e-11,-9.377372e-05
50%,30.000000,15.507625,1.538639e-05,1.538638e-05,1.574474e-14,9.971536e-08
75%,40.000000,23.260438,1.585383e-05,1.585384e-05,1.439578e-11,9.356970e-05
max,50.000000,31.013250,1.660800e-05,1.660800e-05,2.961264e-10,1.836221e-03


## 3 Fluid Isentropic Exponent Fit

For this model, we simply assume that the real gas isentropic exponent can be expressed directly as a linear model of the temperature and pressures of the fluid expressed as a polynomial:

$$\hat\kappa=\Phi(T,P)\vec{\beta}$$

### Hydrogen

In [2]:
"""Isentropic Exponent Fit Example Code with inputs for temperature and pressure in degrees Celsius and bar(a), respectively."""

from fit_fluid_properties import fit_isentropic_exponent

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "HYDROGEN PROPERTIES.xlsx",
    "reference_fluid": "HYDROGEN",
    "model_poly_degree": 4,
}

df_isentropic_exponent, functional_form_text_isentropic_exponent = (
    fit_isentropic_exponent(params)
)

print(functional_form_text_isentropic_exponent)
df_isentropic_exponent.describe()

1.409596891653364 + 0.0*T**0*P**0 + -0.00021168113108057456*T**1*P**0 + 0.0010777875895402658*T**0*P**1 + 1.6796557071098336e-06*T**2*P**0 + -4.287204255942499e-06*T**1*P**1 + 3.413746919573821e-07*T**0*P**2 + -9.7635194589422e-09*T**3*P**0 + 1.9572433326999684e-08*T**2*P**1 + -6.210003209104415e-09*T**1*P**2 + -5.177935277717193e-10*T**0*P**3 + 4.6351078257693556e-11*T**4*P**0 + -7.402860687075777e-11*T**3*P**1 + 1.745268492374971e-11*T**2*P**2 + 1.0364208589388937e-11*T**1*P**3 + -9.0810581168492e-13*T**0*P**4


,Temperature (C),Pressure (bar),Isentropic Exponent,Isentropic Exponent_hat,Isentropic Exponent Error,Isentropic Exponent Relative Error (%)
count,250000.000000,250000.000000,250000.000000,250000.000000,2.500000e+05,2.500000e+05
mean,27.500000,100.507625,1.504841,1.504841,1.104228e-14,1.067330e-09
std,13.016414,58.143236,0.058554,0.058554,4.918117e-06,3.312387e-04
min,5.000000,0.002000,1.402281,1.402283,-1.781480e-05,-1.110519e-03
25%,16.250000,50.254812,1.454203,1.454204,-3.365074e-06,-2.241511e-04
50%,27.500000,100.507625,1.504290,1.504289,4.862555e-08,3.228618e-06
75%,38.750000,150.760437,1.554659,1.554657,3.330730e-06,2.202425e-04
max,50.000000,201.013250,1.628311,1.628306,2.521122e-05,1.793554e-03


### Nitrogen

In [4]:
"""Isentropic Exponent Fit Example Code with inputs for temperature and pressure in degrees Celsius and bar(a), respectively."""

from fit_fluid_properties import fit_isentropic_exponent

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "NITROGEN PROPERTIES.xlsx",
    "reference_fluid": "NITROGEN",
    "model_poly_degree": 6,
}

df_isentropic_exponent, functional_form_text_isentropic_exponent = (
    fit_isentropic_exponent(params)
)

print(functional_form_text_isentropic_exponent)
df_isentropic_exponent.describe()

1.3997549278436698 + -2.2472227522000885e-07*T**0*P**0 + -2.0278868208888922e-05*T**1*P**0 + 0.001492455914059335*T**0*P**1 + 1.2341814765440905e-06*T**2*P**0 + -7.292876566031909e-07*T**1*P**1 + 8.516980837407223e-06*T**0*P**2 + -6.064619470762657e-08*T**3*P**0 + -4.222643865246948e-08*T**2*P**1 + -1.0246405151714761e-07*T**1*P**2 + 1.284319827635135e-08*T**0*P**3 + 1.6829020862005872e-09*T**4*P**0 + 3.800752374561196e-10*T**3*P**1 + 1.2881165190490205e-09*T**2*P**2 + -4.953100593973121e-10*T**1*P**3 + -5.329665123911961e-11*T**0*P**4 + -2.518182337465865e-11*T**5*P**0 + 2.4335366254316976e-12*T**4*P**1 + -1.350659551168495e-11*T**3*P**2 + 3.061332688815223e-12*T**2*P**3 + 1.9664948471311745e-12*T**1*P**4 + -8.712820612399251e-14*T**0*P**5 + 1.497858855555944e-13*T**6*P**0 + -3.1569284856672084e-14*T**5*P**1 + 4.4952878416744816e-14*T**4*P**2 + 9.043665639869587e-15*T**3*P**3 + -1.1815062469716447e-14*T**2*P**4 + -1.1181211833418293e-15*T**1*P**5 + 2.626208712303013e-16*T**0*P**6


,Temperature (C),Pressure (bar),Isentropic Exponent,Isentropic Exponent_hat,Isentropic Exponent Error,Isentropic Exponent Relative Error (%)
count,249999.000000,249999.000000,249999.000000,249999.000000,2.499990e+05,2.499990e+05
mean,27.500048,100.507283,1.629965,1.629965,-1.295768e-14,-4.646283e-07
std,13.016418,58.143100,0.158785,0.158785,3.162270e-06,1.984256e-04
min,5.000000,0.002000,1.399240,1.399237,-1.882367e-05,-1.344919e-03
25%,16.272545,50.154105,1.488415,1.488415,-1.721501e-06,-1.047090e-04
50%,27.545090,100.306211,1.609934,1.609927,1.338721e-07,8.379354e-06
75%,38.772545,150.659730,1.759706,1.759708,1.599594e-06,9.845479e-05
max,50.000000,201.013250,2.023697,2.023700,1.629878e-05,9.186961e-04


### Methane

In [9]:
"""Isentropic Exponent Fit Example Code with inputs for temperature and pressure in degrees Celsius and bar(a), respectively."""

from fit_fluid_properties import fit_isentropic_exponent

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "METHANE PROPERTIES.xlsx",
    "reference_fluid": "METHANE",
    "model_poly_degree": 6,
}

df_isentropic_exponent, functional_form_text_isentropic_exponent = (
    fit_isentropic_exponent(params)
)

print(functional_form_text_isentropic_exponent)
df_isentropic_exponent.describe()

1.312345591546896 + 9.835469070523195e-08*T**0*P**0 + -0.0002727167858678424*T**1*P**0 + 0.0003150515060549049*T**0*P**1 + -1.0183291033294471e-05*T**2*P**0 + 2.7641286998309977e-06*T**1*P**1 + 7.837353116885395e-06*T**0*P**2 + 6.16013845832951e-07*T**3*P**0 + -4.2021065964620497e-07*T**2*P**1 + 3.690918468515584e-07*T**1*P**2 + 5.164591203317966e-08*T**0*P**3 + -1.5551099892985143e-08*T**4*P**0 + 3.4063512875777026e-09*T**3*P**1 + 2.8854236936817055e-09*T**2*P**2 + -9.332001500887936e-09*T**1*P**3 + 1.6777800203945952e-09*T**0*P**4 + 1.4479003607440895e-10*T**5*P**0 + 1.84852764649251e-10*T**4*P**1 + -2.490287148075656e-10*T**3*P**2 + 1.4679191555410515e-10*T**2*P**3 + 3.5186420178898834e-12*T**1*P**4 + -8.015596280024011e-12*T**0*P**5 + -4.2340341403839594e-13*T**6*P**0 + -1.4993771717654407e-12*T**5*P**1 + 7.915760625652369e-13*T**4*P**2 + 2.3925980291843196e-13*T**3*P**3 + -4.155375655655498e-13*T**2*P**4 + 7.088518547094733e-14*T**1*P**5 + 7.503925866098049e-15*T**0*P**6


,Temperature (C),Pressure (bar),Isentropic Exponent,Isentropic Exponent_hat,Isentropic Exponent Error,Isentropic Exponent Relative Error (%)
count,250000.000000,250000.000000,250000.000000,250000.000000,2.500000e+05,250000.000000
mean,27.500000,100.507625,1.577877,1.577877,5.971213e-15,0.000007
std,13.016414,58.143236,0.270491,0.270490,2.134305e-04,0.013344
min,5.000000,0.002000,1.292735,1.291691,-1.202581e-03,-0.080756
25%,16.250000,50.254812,1.348975,1.349049,-1.108054e-04,-0.007092
50%,27.500000,100.507625,1.485338,1.485298,-2.771949e-06,-0.000177
75%,38.750000,150.760437,1.752492,1.752472,1.159969e-04,0.007507
max,50.000000,201.013250,2.569691,2.570123,1.018474e-03,0.070157


### CO2

In [3]:
"""Isentropic Exponent Fit Example Code with inputs for temperature and pressure in degrees Celsius and bar(a), respectively."""

from fit_fluid_properties import fit_isentropic_exponent

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "CO2 PROPERTIES.xlsx",
    "reference_fluid": "CO2",
    "model_poly_degree": 6,
}

df_isentropic_exponent, functional_form_text_isentropic_exponent = (
    fit_isentropic_exponent(params)
)

print(functional_form_text_isentropic_exponent)
df_isentropic_exponent.describe()

1.3007614629674578 + 1.1810299180183611e-09*T**0*P**0 + -0.0005444474428457901*T**1*P**0 + -0.0012527001804253305*T**0*P**1 + 3.7780189479848756e-06*T**2*P**0 + 1.6257881397612285e-05*T**1*P**1 + 1.1911575697689721e-06*T**0*P**2 + -1.328563689718393e-07*T**3*P**0 + -4.9998573708162454e-08*T**2*P**1 + 3.168031256210293e-07*T**1*P**2 + 3.3462982564674293e-09*T**0*P**3 + 3.647977100037436e-09*T**4*P**0 + -7.967201233512167e-10*T**3*P**1 + -1.0911969338500882e-08*T**2*P**2 + 7.504689983872538e-09*T**1*P**3 + -1.6278860340852237e-10*T**0*P**4 + -5.053852059779585e-11*T**5*P**0 + 1.415108051068903e-12*T**4*P**1 + 2.4796641012885226e-10*T**3*P**2 + -2.724442491382375e-10*T**2*P**3 + 6.063662096318167e-11*T**1*P**4 + 2.626996123479907e-11*T**0*P**5 + 2.7384388159856776e-13*T**6*P**0 + 1.4400381386468764e-13*T**5*P**1 + -2.2583870402026127e-12*T**4*P**2 + 2.6467272519698284e-12*T**3*P**3 + -2.2298490636417367e-13*T**2*P**4 + -1.3747945381940851e-12*T**1*P**5 + 5.655374850062692e-13*T**0*P**6


,Temperature (C),Pressure (bar),Isentropic Exponent,Isentropic Exponent_hat,Isentropic Exponent Error,Isentropic Exponent Relative Error (%)
count,250000.000000,250000.000000,250000.000000,250000.000000,2.500000e+05,2.500000e+05
mean,30.000000,15.507625,1.275781,1.275781,-1.582972e-14,7.329410e-11
std,11.570146,8.970117,0.006167,0.006167,6.435025e-07,5.050906e-05
min,10.000000,0.002000,1.267325,1.267324,-3.663005e-06,-2.886504e-04
25%,20.000000,7.754812,1.270539,1.270539,-3.663259e-07,-2.872488e-05
50%,30.000000,15.507625,1.274432,1.274432,-1.725343e-08,-1.353461e-06
75%,40.000000,23.260438,1.279759,1.279759,3.558471e-07,2.788272e-05
max,50.000000,31.013250,1.295591,1.295591,6.118953e-06,4.826368e-04


## 3 Critical Flow Factor (C*) Fit

Similarly to both the density and dynamic viscosity calculations, to enhance the precision of the C* calculations beyond what a naive linear regression model could offer, we separate the C* equation into two parts. The first is an idealized part calculated using the ideal gas Law, $C^*_{ideal}$, and the second is a compressible component calculated using a nonlinear function of temperature and pressure, $f(T, P)$.

The ideal part of the C*, $C^*{ideal}$, is given by:
$$
C^*_{ideal} = \sqrt{ 2 \hat\kappa / (\hat\kappa + 1)^{(\hat\kappa + 1)/(\hat\kappa - 1)}}
$$

Where $\kappa$ is the real gas isentropic exponent calculated before.

Given that we have measurements of $T$ and $P$, we can employ the polynomial model $\Phi(T, P)\vec{\beta}$ to approximate the compressible part of the equation:
$$
f(T, P) \approx \Phi(T, P)\vec{\beta}
$$

Consequently, the final approximation of the viscosity can be expressed as:
$$
\hat{C^*} = C^*_{ideal} \cdot \hat{f}(T, P) = \sqrt{ 2 \hat\kappa / (\hat\kappa + 1)^{(\hat\kappa + 1)/(\hat\kappa - 1)}}\cdot \Phi(T, P)\vec{\beta}
$$

This formulation then allows us to estimate the critical flow factor as a function of pressure, temperature, while incorporating adjustments based on the real gas behaviours from our polynomial model.

### Hydrogen

In [2]:
"""Critical Flow Factor Fit Example Code with inputs for temperature and pressure in degrees Celsius and bar(a), respectively."""

from fit_fluid_properties import fit_critical_flow_factor

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "HYDROGEN PROPERTIES.xlsx",
    "reference_fluid": "HYDROGEN",
    "model_poly_degree": 5,
}

df_critical_flow_factor, functional_form_text_critical_flow_factor = (
    fit_critical_flow_factor(params)
)

print(functional_form_text_critical_flow_factor)
df_critical_flow_factor.describe()

sqrt(1.4096040032266608 + 0.0*T**0*P**0 + -0.00021332951409955693*T**1*P**0 + 0.0010785509134712515*T**0*P**1 + 1.6659595559881486e-06*T**2*P**0 + -4.150758972847252e-06*T**1*P**1 + 2.8308564765690186e-07*T**0*P**2 + -8.042980837940391e-09*T**3*P**0 + 1.7006440764566347e-08*T**2*P**1 + -7.77943346278621e-09*T**1*P**2 + 4.1108588154743466e-10*T**0*P**3 + 3.747646862459922e-11*T**4*P**0 + -9.000178640913979e-11*T**3*P**1 + 5.380242534714488e-11*T**2*P**2 + 1.4146002715845333e-11*T**1*P**3 + -6.299784978957878e-12*T**0*P**4 + -1.7282923159473367e-13*T**5*P**0 + 3.2473790048917725e-13*T**4*P**1 + -9.824120293986218e-14*T**3*P**2 + -9.367399705310225e-14*T**2*P**3 + 3.408365712061212e-15*T**1*P**4 + 1.0542402454488675e-14*T**0*P**5 * (2 / (1.4096040032266608 + 0.0*T**0*P**0 + -0.00021332951409955693*T**1*P**0 + 0.0010785509134712515*T**0*P**1 + 1.6659595559881486e-06*T**2*P**0 + -4.150758972847252e-06*T**1*P**1 + 2.8308564765690186e-07*T**0*P**2 + -8.042980837940391e-09*T**3*P**0 + 1.700644

,Temperature (C),Pressure (bar),Isentropic Exponent_hat,Critical Flow Factor,Critical Flow Factor_hat,Critical Flow Factor Error,Critical Flow Factor Relative Error (%)
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,2.500000e+05,2.500000e+05
mean,27.500000,100.507625,1.504841,0.679142,0.679142,-1.122001e-12,-3.081398e-10
std,13.016414,58.143236,0.058554,0.004359,0.004359,9.172368e-08,1.343844e-05
min,5.000000,0.002000,1.402279,0.671075,0.671074,-9.270431e-07,-1.349053e-04
25%,16.250000,50.254812,1.454204,0.675393,0.675393,-4.582276e-08,-6.743647e-06
50%,27.500000,100.507625,1.504289,0.679206,0.679206,4.112904e-09,6.057095e-07
75%,38.750000,150.760437,1.554659,0.682916,0.682915,5.012062e-08,7.384251e-06
max,50.000000,201.013250,1.628310,0.687367,0.687366,9.377742e-07,1.368048e-04


### Nitrogen

In [7]:
"""Critical Flow Factor Fit Example Code with inputs for temperature and pressure in degrees Celsius and bar(a), respectively."""

from fit_fluid_properties import fit_critical_flow_factor

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "NITROGEN PROPERTIES.xlsx",
    "reference_fluid": "NITROGEN",
    "model_poly_degree": 6,
}

df_critical_flow_factor, functional_form_text_critical_flow_factor = (
    fit_critical_flow_factor(params)
)

print(functional_form_text_critical_flow_factor)
df_critical_flow_factor.describe()

1.3997549278436698 + -2.2472227522000885e-07*T**0*P**0 + -2.0278868208888922e-05*T**1*P**0 + 0.001492455914059335*T**0*P**1 + 1.2341814765440905e-06*T**2*P**0 + -7.292876566031909e-07*T**1*P**1 + 8.516980837407223e-06*T**0*P**2 + -6.064619470762657e-08*T**3*P**0 + -4.222643865246948e-08*T**2*P**1 + -1.0246405151714761e-07*T**1*P**2 + 1.284319827635135e-08*T**0*P**3 + 1.6829020862005872e-09*T**4*P**0 + 3.800752374561196e-10*T**3*P**1 + 1.2881165190490205e-09*T**2*P**2 + -4.953100593973121e-10*T**1*P**3 + -5.329665123911961e-11*T**0*P**4 + -2.518182337465865e-11*T**5*P**0 + 2.4335366254316976e-12*T**4*P**1 + -1.350659551168495e-11*T**3*P**2 + 3.061332688815223e-12*T**2*P**3 + 1.9664948471311745e-12*T**1*P**4 + -8.712820612399251e-14*T**0*P**5 + 1.497858855555944e-13*T**6*P**0 + -3.1569284856672084e-14*T**5*P**1 + 4.4952878416744816e-14*T**4*P**2 + 9.043665639869587e-15*T**3*P**3 + -1.1815062469716447e-14*T**2*P**4 + -1.1181211833418293e-15*T**1*P**5 + 2.626208712303013e-16*T**0*P**6 * (2

,Temperature (C),Pressure (bar),Isentropic Exponent_hat,Critical Flow Factor,Critical Flow Factor_hat,Critical Flow Factor Error,Critical Flow Factor Relative Error (%)
count,249999.000000,249999.000000,249999.000000,249999.000000,249999.000000,2.499990e+05,2.499990e+05
mean,27.500048,100.507283,1.629965,0.705973,0.705973,1.378654e-10,3.126771e-09
std,13.016418,58.143100,0.158785,0.011976,0.011976,2.680449e-07,3.798140e-05
min,5.000000,0.002000,1.399237,0.684657,0.684656,-1.276985e-06,-1.762318e-04
25%,16.272545,50.154105,1.488415,0.696180,0.696180,-1.583631e-07,-2.244556e-05
50%,27.545090,100.306211,1.609927,0.706313,0.706313,6.275899e-09,8.940601e-07
75%,38.772545,150.659730,1.759708,0.714649,0.714650,1.526017e-07,2.167826e-05
max,50.000000,201.013250,2.023700,0.736099,0.736100,3.432613e-06,4.751326e-04


### Methane

In [1]:
"""Critical Flow Factor Fit Example Code with inputs for temperature and pressure in degrees Celsius and bar(a), respectively."""

from fit_fluid_properties import fit_critical_flow_factor

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "METHANE PROPERTIES.xlsx",
    "reference_fluid": "METHANE",
    "model_poly_degree": 6,
}

df_critical_flow_factor, functional_form_text_critical_flow_factor = (
    fit_critical_flow_factor(params)
)

print(functional_form_text_critical_flow_factor)
df_critical_flow_factor.describe()

sqrt(1.312345591546896 + 9.835469070523195e-08*T**0*P**0 + -0.0002727167858678424*T**1*P**0 + 0.0003150515060549049*T**0*P**1 + -1.0183291033294471e-05*T**2*P**0 + 2.7641286998309977e-06*T**1*P**1 + 7.837353116885395e-06*T**0*P**2 + 6.16013845832951e-07*T**3*P**0 + -4.2021065964620497e-07*T**2*P**1 + 3.690918468515584e-07*T**1*P**2 + 5.164591203317966e-08*T**0*P**3 + -1.5551099892985143e-08*T**4*P**0 + 3.4063512875777026e-09*T**3*P**1 + 2.8854236936817055e-09*T**2*P**2 + -9.332001500887936e-09*T**1*P**3 + 1.6777800203945952e-09*T**0*P**4 + 1.4479003607440895e-10*T**5*P**0 + 1.84852764649251e-10*T**4*P**1 + -2.490287148075656e-10*T**3*P**2 + 1.4679191555410515e-10*T**2*P**3 + 3.5186420178898834e-12*T**1*P**4 + -8.015596280024011e-12*T**0*P**5 + -4.2340341403839594e-13*T**6*P**0 + -1.4993771717654407e-12*T**5*P**1 + 7.915760625652369e-13*T**4*P**2 + 2.3925980291843196e-13*T**3*P**3 + -4.155375655655498e-13*T**2*P**4 + 7.088518547094733e-14*T**1*P**5 + 7.503925866098049e-15*T**0*P**6 * (2

,Temperature (C),Pressure (bar),Isentropic Exponent_hat,Critical Flow Factor,Critical Flow Factor_hat,Critical Flow Factor Error,Critical Flow Factor Relative Error (%)
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,2.500000e+05,2.500000e+05
mean,27.500000,100.507625,1.577877,0.747314,0.747314,-6.097783e-09,2.874435e-07
std,13.016414,58.143236,0.270490,0.052180,0.052179,1.375731e-05,1.850988e-03
min,5.000000,0.002000,1.291691,0.667791,0.667649,-1.422120e-04,-2.129587e-02
25%,16.250000,50.254812,1.349049,0.702324,0.702326,-9.952046e-06,-1.325004e-03
50%,27.500000,100.507625,1.485298,0.742516,0.742515,2.636507e-07,3.537909e-05
75%,38.750000,150.760437,1.752472,0.784679,0.784684,9.621590e-06,1.271210e-03
max,50.000000,201.013250,2.570123,0.894630,0.894768,1.386703e-04,1.550030e-02


### CO2

In [4]:
"""Critical Flow Factor Fit Example Code with inputs for temperature and pressure in degrees Celsius and bar(a), respectively."""

from fit_fluid_properties import fit_critical_flow_factor

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "CO2 PROPERTIES.xlsx",
    "reference_fluid": "CO2",
    "model_poly_degree": 6,
}

df_critical_flow_factor, functional_form_text_critical_flow_factor = (
    fit_critical_flow_factor(params)
)

print(functional_form_text_critical_flow_factor)
df_critical_flow_factor.describe()

sqrt(1.3007614629674578 + 1.1810299180183611e-09*T**0*P**0 + -0.0005444474428457901*T**1*P**0 + -0.0012527001804253305*T**0*P**1 + 3.7780189479848756e-06*T**2*P**0 + 1.6257881397612285e-05*T**1*P**1 + 1.1911575697689721e-06*T**0*P**2 + -1.328563689718393e-07*T**3*P**0 + -4.9998573708162454e-08*T**2*P**1 + 3.168031256210293e-07*T**1*P**2 + 3.3462982564674293e-09*T**0*P**3 + 3.647977100037436e-09*T**4*P**0 + -7.967201233512167e-10*T**3*P**1 + -1.0911969338500882e-08*T**2*P**2 + 7.504689983872538e-09*T**1*P**3 + -1.6278860340852237e-10*T**0*P**4 + -5.053852059779585e-11*T**5*P**0 + 1.415108051068903e-12*T**4*P**1 + 2.4796641012885226e-10*T**3*P**2 + -2.724442491382375e-10*T**2*P**3 + 6.063662096318167e-11*T**1*P**4 + 2.626996123479907e-11*T**0*P**5 + 2.7384388159856776e-13*T**6*P**0 + 1.4400381386468764e-13*T**5*P**1 + -2.2583870402026127e-12*T**4*P**2 + 2.6467272519698284e-12*T**3*P**3 + -2.2298490636417367e-13*T**2*P**4 + -1.3747945381940851e-12*T**1*P**5 + 5.655374850062692e-13*T**0*P*

,Temperature (C),Pressure (bar),Isentropic Exponent_hat,Critical Flow Factor,Critical Flow Factor_hat,Critical Flow Factor Error,Critical Flow Factor Relative Error (%)
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,2.500000e+05,2.500000e+05
mean,30.000000,15.507625,1.275781,0.693484,0.693484,3.758856e-13,1.153103e-09
std,11.570146,8.970117,0.006167,0.018068,0.018068,2.619578e-06,3.696243e-04
min,10.000000,0.002000,1.267324,0.665011,0.665039,-5.813181e-05,-7.756042e-03
25%,20.000000,7.754812,1.270539,0.678197,0.678197,-1.497157e-06,-2.159593e-04
50%,30.000000,15.507625,1.274432,0.691384,0.691384,-5.048473e-09,-7.191344e-07
75%,40.000000,23.260438,1.279759,0.706525,0.706525,1.413241e-06,2.046757e-04
max,50.000000,31.013250,1.295591,0.749504,0.749445,2.886405e-05,4.340390e-03
